# Imports

In [1]:
# import pandas as pd
# import numpy as np
# import os
# import unidecode
# import datetime
# # from Utilities import *

# import warnings
# warnings.simplefilter(action="ignore")

# baseball_path = r"C:\Users\james\Documents\MLB\Data2"

In [2]:
# # Today's Date
# # YYYY-MM-DD (datetime)
# todaysdate_dt = datetime.date.today()

# # YYYY-MM-DD (string)
# todaysdate_dash = str(todaysdate_dt)

# # MM/DD/YYYY
# todaysdate_slash = todaysdate_dash.split("-")
# todaysdate_slash = todaysdate_slash[1] + "/" + todaysdate_slash[2] + "/" + todaysdate_slash[0]

# # YYYYMMDD
# todaysdate = todaysdate_dash.replace("-", "")

In [1]:
# # Read in history file        
# history = pd.read_csv(os.path.join(baseball_path, "Utilities", "Contests.csv"))

# # Sort by date, then draft group, then fee
# history.sort_values(['date', 'draftGroupId', 'entryFee'], ascending=False)
# # Keep only one observation per draft group 
# history.drop_duplicates('draftGroupId', keep='first', inplace=True)

# history_full = history.copy()
# history = history.query('result == 1').query('payout == 1').query('salary == 1')


# history = history.reset_index(drop=True)
# history['date'] = history['date'].astype('int').astype('str')

In [4]:
# %run "RotoWire.ipynb"
# %run "DFF.ipynb"
# %run "Utilities.ipynb"

In [6]:
# Read in my projections
def read_player_sims(contestKey):
    # Read in projections
    my_df = pd.read_csv(os.path.join(baseball_path, "10. Sims", "A. Players", "Player Sims " + str(contestKey) + ".csv"), encoding='iso-8859-1')
    
    # Remove accents
    my_df['Name'] = my_df['Name'].apply(remove_accents)
    
    # Remove suffixes
    my_df['Name'] = my_df['Name'].str.replace(r'\s+(Jr\.|Sr\.|II|III)$', '', regex=True)

    my_df['lastName'] = my_df['Name'].str.split(" ").str[-1]
    
    
    # Remove low scores (relief pitchers, mostly)
    my_df = my_df[my_df['AvgPointsPerGame'] > 4]

    # Rename for consistency
    my_df.rename(columns={'AvgPointsPerGame': 'Projection_Me'}, inplace=True)
    
    # Keep relevant variables
    my_df = my_df[['Name', 'lastName', 'TeamAbbrev', 'Salary', 'Projection_Me']]
    
    return my_df

In [7]:
def merge_projections(contestKey):
    # Read in my projections
    my_projections = read_player_sims(contestKey)    
    
    # Read in RotoWire's projections
    rotowire_projections = read_roto(contestKey)
    rotowire_projections['Name'] = rotowire_projections['firstName'] + " " + rotowire_projections['lastName']
    rotowire_projections.rename(columns={'teamAbbr':'TeamAbbrev', 'salary': 'Salary'}, inplace=True)    
    
    # Merge using Name and Team
    projection_df = pd.merge(my_projections, rotowire_projections[['Name', 'TeamAbbrev', 'rostership', 'points']], on=['Name', 'TeamAbbrev'], how='left', validate='one_to_one', indicator=True)
    # Rename merge indicator (to allow for another)
    projection_df.rename(columns={'_merge': '_merge1'}, inplace=True)    
    # Merge using Last Name, Team, and Salary
    projection_df = pd.merge(projection_df, rotowire_projections[['lastName', 'TeamAbbrev', 'Salary', 'rostership', 'points']], on=['lastName', 'TeamAbbrev', 'Salary'], how='left', indicator=True)
    # Drop duplicates
    projection_df.drop_duplicates(['Name', 'TeamAbbrev'], inplace=True)
    
    # Fill in missing points
    projection_df['points_x'].fillna(projection_df['points_y'], inplace=True)
    projection_df['rostership_x'].fillna(projection_df['rostership_y'], inplace=True)
    
    # Identify those that didn't merge
    print("RotoWire Didn't Merge:")
    print(projection_df[(projection_df['_merge1'] == "left_only") & (projection_df['_merge'] == "left_only")]['Name'])
    
    # Rename column
    projection_df.rename(columns={'points_x':'Projection_Roto', 'rostership_x':'ownership'}, inplace=True)

    # Keep relevant variables
    projection_df = projection_df[['Name', 'lastName', 'TeamAbbrev', 'Salary', 'ownership', 'Projection_Me', 'Projection_Roto']]
    
    
    ### DFF
    # Read in DFF's projections
    dff_projections = read_dff(contestKey)
    
    # If it's the old version (merged on date, downloaded csv), rename to match
    try:
        dff_projections.rename(columns={'first_name': 'First Name', 'last_name': 'Last Name', 'team': 'Team', 'salary': 'Salary', 'ppg_projection': 'FP'}, inplace=True)
    except:
        pass
    
    # Create Name variable
    dff_projections['Name'] = dff_projections['First Name'] + " " + dff_projections['Last Name']
    dff_projections['Name'] = dff_projections['Name'].str.replace(r'\s+(Jr\.|Sr\.|II|III)$', '', regex=True)
    dff_projections.rename(columns={'Team':'TeamAbbrev', 'Last Name': 'lastName'}, inplace=True)
    
    # Merge using Name and Team
    projection_df = pd.merge(projection_df, dff_projections[['Name', 'TeamAbbrev', 'FP']], on=['Name', 'TeamAbbrev'], how='left', validate='one_to_one', indicator=True)
    # Rename merge indicator (to allow for another)
    projection_df.rename(columns={'_merge': '_merge1'}, inplace=True)
    # Merge using Last Name, Team, and Salary
    projection_df = pd.merge(projection_df, dff_projections[['lastName', 'TeamAbbrev', 'Salary', 'FP']], on=['lastName', 'TeamAbbrev', 'Salary'], how='left', indicator=True)
    # Drop duplicates
    projection_df.drop_duplicates(['Name', 'TeamAbbrev'], inplace=True)
    
    # Fill in missing points
    projection_df['FP_x'].fillna(projection_df['FP_y'], inplace=True)    
    
    # Identify those that didn't merge
    print("DFF Didn't Merge:")
    print(projection_df[(projection_df['_merge1'] == "left_only") & (projection_df['_merge'] == "left_only")]['Name'])
    
    # Rename column
    projection_df.rename(columns={'FP_x':'Projection_DFF'}, inplace=True)
    
    # Keep relevant variables
    projection_df = projection_df[['Name', 'lastName', 'TeamAbbrev', 'Salary', 'ownership', 'Projection_Me', 'Projection_Roto', 'Projection_DFF']]
    
    
    return projection_df

In [8]:
def evaluate_projections_pre(daily):
    ### Correlations
    # My projections with DFF
    dff_corr = daily['Projection_Me'].corr(daily['Projection_DFF'])
    # My projections with RotoWire
    roto_corr = daily['Projection_Me'].corr(daily['Projection_Roto'])
    # DFF's projections with Rotowire's
    their_corr = daily['Projection_DFF'].corr(daily['Projection_Roto'])
    
    # print(dff_corr)
    # print(roto_corr)
    # print(their_corr)
    
    ### Outliers
    # DFF
    daily['Diff_DFF'] = daily['Projection_Me'] - daily['Projection_DFF']
    daily.sort_values(by='Diff_DFF', ascending=True, inplace=True)
    
    print("I'm lower than DFF on:")
    print(daily[['Name', 'Projection_Me', 'Projection_DFF']].head(10))

    daily.sort_values(by='Diff_DFF', ascending=False, inplace=True)
    
    print("I'm higher than DFF on:")
    print(daily[['Name', 'Projection_Me', 'Projection_DFF']].head(10))

    
    # RotoWire
    daily['Diff_Roto'] = daily['Projection_Me'] - daily['Projection_Roto']
    daily.sort_values(by='Diff_Roto', ascending=True, inplace=True)
    
    print("I'm lower than RotoWire on:")
    print(daily[['Name', 'Projection_Me', 'Projection_Roto', 'ownership']].head(10))

    daily.sort_values(by='Diff_Roto', ascending=False, inplace=True)
    
    print("I'm higher than RotoWire on:")
    print(daily[['Name', 'Projection_Me', 'Projection_Roto', 'ownership']].head(10))

    return daily

# After the day

In [9]:
# Read in results
def read_results(contestKey):
    score_df = pd.read_csv(os.path.join(baseball_path, "1. Results", "A. Contest Results", "contest-standings-" + str(contestKey) + ".csv"))
    score_df = score_df[['Player', 'Roster Position', '%Drafted', 'FPTS']]
    score_df.dropna(inplace=True)
    score_df.rename(columns={'Player':'Name'}, inplace=True)
    
    # Remove accents
    score_df['Name'] = score_df['Name'].apply(remove_accents)
    
    # Remove suffixes
    score_df['Name'] = score_df['Name'].str.replace(r'\s+(Jr\.|Sr\.|II|III)$', '', regex=True)
    
    # Replace pitcher 0s with missing
    score_df['FPTS'] = np.where(((score_df['FPTS'] == 0) & (score_df['Roster Position'] == "P")), np.nan, score_df['FPTS'])

    score_df = score_df[['Name', 'FPTS', '%Drafted', 'Roster Position']]
    
    score_df.dropna(inplace=True)             
    return score_df

In [10]:
def evaluate_projections_post(contestKey):
    # Read in my projections and others
    projection_df = merge_projections(contestKey)
    # Read in actual scores
    score_df = read_results(contestKey)
    
    # Merge projections with actual scores, keeping only if player was projected
    complete_df = projection_df.merge(score_df, on=['Name'], how='inner')
    
    # My errors
    complete_df['error_Me'] = complete_df['Projection_Me'] - complete_df['FPTS']
    complete_df['MSE_Me'] = complete_df['error_Me'] ** 2
    
    # DFF's errors
    complete_df['error_DFF'] = complete_df['Projection_DFF'] - complete_df['FPTS']
    complete_df['MSE_DFF'] = complete_df['error_DFF'] ** 2
    
    # RotoWire's errors
    complete_df['error_Roto'] = complete_df['Projection_Roto'] - complete_df['FPTS']
    complete_df['MSE_Roto'] = complete_df['error_Roto'] ** 2
    
    # Closer on player 
    complete_df['beat_DFF'] = (complete_df['MSE_DFF'] > complete_df['MSE_Me']).astype('int')
    complete_df['beat_Roto'] = (complete_df['MSE_Roto'] > complete_df['MSE_Me']).astype('int')    
                               
    # May want to add RotoWire's ownership error
    
    return complete_df

In [11]:
# Compare my projections to DFF's and RotoWire's based on actual results
def compare_projections(complete_df):
    # Just look at ones where we all have projections
    comparable_df = complete_df[(~complete_df['Projection_Me'].isna()) & (~complete_df['Projection_DFF'].isna()) & (~complete_df['Projection_Roto'].isna())]
    
    # Average MSEs
    MSE_Me = comparable_df['MSE_Me'].mean()
    MSE_DFF = comparable_df['MSE_DFF'].mean()
    MSE_Roto = comparable_df['MSE_Roto'].mean()
    
    # Create dictionary with variable names and values
    mse_dict = {'MSE_Me': MSE_Me, 'MSE_DFF': MSE_DFF, 'MSE_Roto': MSE_Roto}

    # Sort dictionary by values
    sorted_mse_dict = dict(sorted(mse_dict.items(), key=lambda item: item[1]))

    # Extract names and values in order
    mse_names = list(sorted_mse_dict.keys())
    mse_values = list(sorted_mse_dict.values())

    print("Rankings:")
    for i in range(len(mse_names)):
        print(f"{i+1}. {mse_names[i]}: {mse_values[i]}")
        
    # Separate batters and pitchers
    batter_df = comparable_df[~comparable_df['Roster Position'].isin(['SP', 'RP', 'P'])]
    pitcher_df = comparable_df[comparable_df['Roster Position'].isin(['SP', 'RP', 'P'])]
        
    # Rate at beating other projections
    beat_DFF = comparable_df['beat_DFF'].mean() * 100
    beat_Roto = comparable_df['beat_Roto'].mean() * 100
    
    beat_DFF_B = batter_df['beat_DFF'].mean() * 100
    beat_DFF_P = pitcher_df['beat_DFF'].mean() * 100
    
    beat_Roto_B = batter_df['beat_Roto'].mean() * 100
    beat_Roto_P = pitcher_df['beat_Roto'].mean() * 100
    
    print("There were {} players we all projected.".format(len(comparable_df)))
    print("I beat DFF on {:2.1f}% of players (Batters: {:2.1f}%, Pitchers: {:2.1f}%).".format(beat_DFF, beat_DFF_B, beat_DFF_P))
    print("I beat RotoWire on {:2.1f}% of players (Batters: {:2.1f}%, Pitchers: {:2.1f}%).".format(beat_Roto, beat_Roto_B, beat_Roto_P))
    print("\n")
    
    
    
    return comparable_df

### Run

In [1]:
# # Run all
# history.reset_index(drop=True, inplace=True)
# daily_comparable_dfs = []
# daily_complete_dfs = []
# for i in range(len(history)): 
#     # Extract date and contestKey
#     date = str(history['date'][i])
#     date = date.replace(".0", "")
#     contestKey = history['contestKey'][i]
#     print(date, contestKey)
#     if date <= "920230705":
#         try:
#             # Read in projection evaluations
#             complete_df = evaluate_projections_post(contestKey)
#             daily_complete_dfs.append(complete_df)
#             # Make comparable
#             comparable_df = compare_projections(complete_df)
#             if len(comparable_df) > 30:
#                 daily_comparable_dfs.append(comparable_df)
#         except:
#             pass

# print("Complete")
# all_df = pd.concat(daily_comparable_dfs, axis=0)
# all_df = compare_projections(all_df)


# all_df_complete = pd.concat(daily_complete_dfs, axis=0)

In [13]:
# print("Batters")
# print(all_df_complete[all_df_complete['Roster Position'] != "P"].query('Projection_Me > 5').query('Projection_DFF > 5')[['Projection_Me', 'Projection_DFF', 'Projection_Roto', 'FPTS']].describe())
# print("\nPitchers")
# print(all_df_complete[all_df_complete['Roster Position'] == "P"].query('Projection_Me > 5').query('Projection_DFF > 5')[['Projection_Me', 'Projection_DFF', 'Projection_Roto', 'FPTS']].describe())